In [ ]:
import numpy as np
import pandas as pd
import volpriceinference as vl
import matplotlib as mpl
import seaborn as sns
import logging
import sys

In [ ]:
%matplotlib inline
mpl.style.use('seaborn-talk')
np.set_printoptions(precision=3)

In [ ]:
logging.captureWarnings(True)
root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)
info_handler = logging.FileHandler('../logging.tmp.txt')
info_handler.addFilter(lambda record: record.levelno <= logging.WARNING)
error_handler = logging.StreamHandler(sys.stderr)
error_handler.addFilter(lambda record: record.levelno > logging.WARNING)
root_logger.addHandler(info_handler)
root_logger.addHandler(error_handler)

In [ ]:
bounds = {'pi': {'max': 0, 'min':-14.5, 'dim':5}, 'phi': {'max': -.25, 'min':-.35, 'dim':3}, 
          'theta': {'max':2.5, 'min': 0, 'dim': 5}}
innov_dim = 500

In [ ]:
vl.__version__

## Load the data

In [ ]:
with pd.HDFStore('../data/spy_daily_estimates_2003-2017_1000_1sec.hdf') as store:
    daily_rtn = store['daily_rtn'] * 40
    vol = store['discrete_vol_est'] * 40**2
    
    data = pd.concat([daily_rtn, vol.diffusion + vol.jumps], axis=1).dropna()
    data.columns = ['rtn', 'vol']

In [ ]:
data.head()

In [ ]:
data.transform(lambda x: np.sqrt(x) if x.name == 'vol' else x).cov()

In [ ]:
data.corr()

In [ ]:
data.describe()

In [ ]:
data.describe()

In [ ]:
fig1, ax1 = mpl.pyplot.subplots()
data['rtn'].plot(ax=ax1, color='blue', alpha=.5, label="$r_{t+1}$")
data['vol'].apply(np.sqrt).plot(ax=ax1, label='$\sqrt{\sigma^2_{t+1}}$', color='red')
ax1.legend()
fig1.savefig('../doc/figures/time_series.tmp.pdf', frameon=False, pad_inches=0, bbox_inches='tight',
            transparent=True)

In [ ]:
joint_grid = sns.JointGrid(x='vol', y='rtn', data=data, xlim=[-.1, 9], ylim=[-3.5, 3.5])
joint_grid.plot_joint(sns.regplot, ci=None, color='purple', scatter_kws={"alpha":.5})
sns.kdeplot(data['vol'], ax=joint_grid.ax_marg_x, legend=None, shade=True, color='red')
sns.kdeplot(data['rtn'], ax=joint_grid.ax_marg_y, legend=None, shade=True, color='blue', vertical=True)
joint_grid.set_axis_labels("$\sigma^2_{t+1}$","$r_{t+1}$")
joint_grid.savefig('../doc/figures/joint_dist.tmp.pdf', frameon=False, pad_inches=0, bbox_inches='tight',
            transparent=True),

In [ ]:
omega, omega_cov = vl.estimate_params_strong_id(data)

In [ ]:
estimates, covariance = vl.estimate_params(data, *vl.compute_vol_gmm(data.vol))

In [ ]:
estimates

In [ ]:
covariance

In [ ]:
vl.compute_strong_id(estimates, covariance)

In [ ]:
strong_id_results = vl.estimate_params_strong_id(data) 

In [ ]:
strong_id_results

In [ ]:
pd.Series(omega)

In [ ]:
omega_cov

In [ ]:
data.vol.sort_values(ascending=True).head()

In [ ]:
omega2, omega_cov2 = vl.estimate_params(data)

In [ ]:
omega2

In [ ]:
omega

In [ ]:
qlr_draws = vl.compute_qlr_sim(omega=omega2, omega_cov=omega_cov2,   theta_dim=bounds['theta']['dim'],
                               pi_dim=bounds['pi']['dim'], phi_dim=bounds['phi']['dim'],
                               pi_min=bounds['pi']['min'], pi_max=bounds['pi']['max'], 
                               phi_min=bounds['phi']['min'], phi_max=bounds['phi']['max'],
                               theta_min=bounds['theta']['min'], theta_max=bounds['theta']['max'],
                               use_tqdm=True, alpha=0.05, innov_dim=innov_dim)

In [ ]:
qlr_stats = vl.compute_qlr_stats(omega=omega2, omega_cov=omega_cov2, theta_dim=bounds['theta']['dim'],
                               pi_dim=bounds['pi']['dim'], phi_dim=bounds['phi']['dim'],
                               pi_min=bounds['pi']['min'], pi_max=bounds['pi']['max'], 
                               phi_min=bounds['phi']['min'], phi_max=bounds['phi']['max'],
                               theta_min=bounds['theta']['min'], theta_max=bounds['theta']['max'],
                                 use_tqdm=True)

In [ ]:
qlr_draws

In [ ]:
qlr_stats

In [ ]:
bounds

In [ ]:
qlr_stats.set_index(['phi', 'pi', 'theta'])

In [ ]:
qlr_stats.count()

In [ ]:
qlr_stats.sort_values(by='qlr').head()

In [ ]:
qlr_draws.sort_values(by='qlr').head()

In [ ]:
merged_values = vl.merge_draws_and_sims(qlr_stats, qlr_draws)

In [ ]:
accepted_vals = merged_values.query('qlr_stats > qlr_draws').drop(['qlr_draws', 'qlr_stats'], axis='columns')

In [ ]:
qlr_draws.to_json(f'../results/qlr_draws_on_data_{innov_dim}_smaller_region_flattened.tmp.json')

In [ ]:
accepted_vals

In [ ]:
qlr_draws = pd.read_json(f'../results/qlr_draws_on_data_{innov_dim}_smaller_region_flattened.tmp.json')

In [ ]:
qlr_draws

In [ ]:
def constraint(scale, psi, zeta, equity_price):
    
    vol_price  = -1 / scale - ((psi * (equity_price-1) + zeta/2 * (equity_price-1)**2))
    
    return  vol_price

In [ ]:
accepted_vals

In [ ]:
omega

In [ ]:
accepted_vals

In [ ]:
accepted_vals

In [ ]:
fig3, ax3 = mpl.pyplot.subplots()
ax3.scatter(y=accepted_vals.theta, x=accepted_vals.pi, marker='o', s=650, label='Not Rejected Values',
           color='purple')

max_y = bounds['theta']['max']
min_x = bounds['pi']['min']

yvals = np.linspace(0,max_y+1,100)
xvals = [constraint(np.exp(omega['log_scale']), omega['psi'], omega['zeta'], x) for x in yvals]
ax3.plot(xvals, yvals, color='black', label='Constraint')
ax3.axvline(0,  ymin=0, ymax=1, color='black')
ax3.axhline(0,  xmin=1, xmax=0, color='black')

white_rect1 = mpl.patches.Rectangle((0,-1), 2, max_y + 2, angle=0.0, color='white', fill=True)
white_rect2 = mpl.patches.Rectangle((min_x-1,-1), abs(min_x) + 2, 1, angle=0.0, color='white', fill=True)
ax3.add_patch(white_rect1)
ax3.add_patch(white_rect2)
ax3.set_ylim([-.2, max_y+.5])
ax3.set_xlim([min_x - .2, .2])
ax3.set_ylabel(r'$\theta$')
ax3.set_xlabel(r'$\pi$')
ax3.legend(loc='upper center')

fig3.savefig(f'../doc/figures/confidence_region_{innov_dim}.tmp.pdf', frameon=False, pad_inches=0, 
             bbox_inches='tight', transparent=True)

In [ ]:
accepted_vals.min()

In [ ]:
accepted_vals.max()

In [ ]:
qlr_draws_pivoted = qlr_draws[qlr_draws.phi == accepted_vals.iloc[0].phi].drop(columns=['phi']).pivot(
    columns='pi',index='theta', values='qlr')

In [ ]:
sns.heatmap(qlr_draws_pivoted, annot=True, cmap=mpl.cm.get_cmap('terrain'), vmin=0, vmax=50)

In [ ]:
qlr_stats

In [ ]:
qlr_stats.phi.unique()

In [ ]:
qlr_stats_pivoted = qlr_stats[qlr_stats.phi ==-0.3].drop(columns=['phi']).pivot(
    columns='pi',index='theta', values='qlr').sort_index(axis='index')

In [ ]:
qlr_stats.query('qlr <= 30').max()

In [ ]:
qlr_stats_pivoted

In [ ]:
qlr_stats_pivoted.query('theta == 2.25')

In [ ]:
sns.heatmap(qlr_stats_pivoted, annot=False, cmap=mpl.cm.get_cmap('terrain'), vmin=0, vmax=50)